## Aggregate locations of dense [AIS broadcast reports](https://marinecadastre.gov/ais) at the port of Miami, FL.

In [ ]:
gdb_path = os.path.join("data", "Miami.gdb")

In [ ]:
df = spark.read.format("gdb").options(path=gdb_path).load().show(truncate=False)

### Create a Spark Dataframe and register it as a table for upcoming SQL operations

In [ ]:
df = spark.read.format("gdb").options(path=gdb_path, name="Broadcast").load()

In [ ]:
df.printSchema()

In [ ]:
df.schema["Shape"].metadata

In [ ]:
df.createOrReplaceTempView("Broadcast")

### Create temp table `QR` by mapping a `Broadcast` event to a cell location.  The cell size is 0.001 degrees.

In [ ]:
cell_1 = 0.001
cell_2 = cell_1 * 0.5

sql(
    f"""
select
cast(floor(Shape.x/{cell_1}D) as INT) as q,
cast(floor(Shape.y/{cell_1}D) as INT) as r
from Broadcast
"""
).createOrReplaceTempView("QR")

### Aggregate by cell and report back only the cells with more that 100 `Broadcast` events.

In [ ]:
sql(
    f"""
select
q*{cell_1}D+{cell_2}D as x,
r*{cell_1}D+{cell_2}D as y,
count(1) as pop
from QR
group by q,r
having pop > 100
"""
).show()